# Ransac algorithm implementation

In [1]:
# libraries
import open3d as o3d
import numpy as np
import random

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [11]:
def ransac(pcd, threshold=0.02
           , iterations=1000):
    # converting the point cloud into numpy array
    points = np.asarray(pcd.points)
    
    best_inliers = []
    best_plane = None
    
    for i in range(iterations):
        
        # randomly sampling 3 points
        random_points_index = random.sample(range(len(points)), 3)
        random_points = points[random_points_index]
        
        # plane - ax + by + cz + d = 0
        p1, p2, p3 = random_points
        
        # computing the normal
        v1 = p2 - p1
        v2 = p3 - p1
        normal = np.cross(v1, v2)
        
        # plane d value
        d = -np.dot(normal, p1)
        
        #Calculating the distance of points from the plane
        distances = np.abs(np.dot(points, normal) + d) / np.linalg.norm(normal)
        
        # inlier definition
        inliers = np.where(distances < threshold)[0]
        
        if len(inliers) > len(best_inliers):
            best_inliers = inliers
            best_plane = (normal[0], normal[1], normal[2], d)
    
    return best_plane, best_inliers

In [13]:
pcd_point_cloud = o3d.data.PCDPointCloud()
pcd = o3d.io.read_point_cloud(pcd_point_cloud.path)

# Visualize the point cloud
o3d.visualization.draw_geometries([pcd],
                                  zoom=1,
                                  front=[0.4257, -0.2125, -0.8795],
                                  lookat=[2.6172, 2.0475, 1.532],
                                  up=[-0.0694, -0.9768, 0.2024])

In [14]:
# applying ransac to the pcd
best_plane, inliers = ransac(pcd)

print("Best fitting plane equation: ax + by + cz + d = 0")
print(f"a = {best_plane[0]}, b = {best_plane[1]}, c = {best_plane[2]}, d = {best_plane[3]}")
print(f"Number of inliers: {len(inliers)}")

# Visualize inliers and outliers
inlier_cloud = pcd.select_by_index(inliers)
outlier_cloud = pcd.select_by_index(list(set(range(len(pcd.points))) - set(inliers)))

# Visualize inliers
inlier_cloud.paint_uniform_color([1.0, 0.0, 0.0])  # Red color for inliers

o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud])


Best fitting plane equation: ax + by + cz + d = 0
a = 0.03409793134778738, b = 0.06956479480919597, c = -0.7501284591853619, d = 0.8231929653913339
Number of inliers: 81378
